In [12]:
import requests
from bs4 import BeautifulSoup

In [1]:
from src.utils.wa_utils import (
    search_wa_athletes,                                     # WA autocomplete (fallback)
    fetch_and_store_wa_results                              # WA scraping (fallback)
)

from src.utils.scraping_wa import (
    search_athletes_by_name as _wa_search,
    get_athlete_results_by_name as _wa_results,
)

c:\Users\Lucas\Documents\DATA_SCIENCE\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
search_wa_athletes("vaudois")

[{'hactseq': None,
  'name': 'Aurelien VAUDOIS',
  'club': 'FRA',
  'sex': 'Men',
  'seq': 'WA_14812063',
  'source': 'WA',
  'aa_id': 14812063}]

In [3]:
_wa_search('jimmy soudril')

,aaAthleteId,familyName,givenName,birthDate,disciplines,iaafId,gender,country,urlSlug,__typename
0,14642716,SOUDRIL,Jimy,29 APR 1998,"4x400 Metres Relay, 400 Metres, 200 Metres",301381,Men,FRA,france/jimy-soudril-14642716,AthleteSearchResult


In [4]:
from src.utils.http_utils import search_athletes            # FFA autocomplete

In [25]:
results = search_athletes("pocreaux")
results

[{'name': 'POCREAUX Cyril',
  'club': ' - Athle 92*',
  'sex': 'M',
  'seq': '124093'},
 {'name': 'POCREAUX Florian',
  'club': ' - Ua Versailles',
  'sex': 'M',
  'seq': '2701495'},
 {'name': 'POCREAUX Raphael',
  'club': ' - Ua Versailles',
  'sex': 'M',
  'seq': '1884739'}]

In [26]:
results = search_athletes("pocreaux")
seq = results[0].get('seq')
# URL à scraper
url=f"https://www.athle.fr/athletes/{seq}/resultats"
# Envoyer la requête au site web
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(url, headers=headers)

# Vérifier si la requête a réussi
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # 1. Chercher un span qui contient "Né(e)" (couvre "Né(e) en" et "Né(e) le")
    label_span = soup.find('span', string=lambda t: t and "Né(e)" in t)
    
    if label_span:
        # 2. Récupérer le texte brut qui suit
        # .next_sibling récupère le texte immédiatement après la balise fermante </span>
        raw_text = label_span.next_sibling
        
        if raw_text:
            # Nettoyage : on enlève les espaces et sauts de ligne
            full_text = raw_text.strip()
            
            # Si on veut juste la date (ex: "19/07/1993") et ignorer le lieu ("à Iloul (MAR)")
            # On prend le premier "mot" qui ressemble à une date
            parts = full_text.split(' ')
            date_naissance = parts[0] # Prend "1998" ou "19/07/1993"
            
            print(f"Texte complet : {full_text}")
            print(f"Date extraite : {date_naissance}")
        else:
            # Fallback : parfois c'est dans une balise <b> juste après
            next_tag = label_span.find_next_sibling('b')
            if next_tag:
                print(f"Date (via tag) : {next_tag.text.strip()}")
            else:
                print("Date introuvable après le label")
    else:
        print("Label 'Né(e)...' non trouvé.")
else:
    print(f"Échec de récupération. Code: {response.status_code}")

Texte complet : 1973
Date extraite : 1973


In [78]:
results = search_athletes("habz")
seq = results[0].get('seq')
# URL à scraper
url = f"https://bases.athle.fr/asp.net/athletes.aspx?base=bilans&seq={seq}"

# Send request to the website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(url, headers=headers)

# Check if request was successful
if response.status_code == 200:
    # Use regex to directly extract the year from the HTML
    match = re.search(r'<td style="width:40%"><b>(\d{4})</b></td>', response.text)
    if match:
        birth_year = match.group(1)
        print("Birth year:", birth_year)
    else:
        print("Birth year not found")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Birth year not found


In [80]:
results = search_athletes("habz")
seq = results[0].get('seq')
# URL à scraper
url = f"https://bases.athle.fr/asp.net/athletes.aspx?base=bilans&seq={seq}"

# Send request to the website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(url, headers=headers)

# Check if request was successful
if response.status_code == 200:
    # Use regex to directly extract date/year from the HTML - handles both "1998" and "19/07/1993" formats
    match = re.search(r'<td style="width:40%"><b>([^<]+)</b></td>', response.text)
    if match:
        birth_date = match.group(1)
        print("Birth date:", birth_date)
    else:
        print("Birth date not found")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Birth date: 19/07/1993


In [85]:
import requests
import re

def scrape_birth_date(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # Use stream=True to stop downloading once we find our match
    with requests.get(url, headers=headers, stream=True) as response:
        if response.status_code != 200:
            return f"Failed to retrieve the webpage. Status code: {response.status_code}"
        
        # Initialize buffer and pattern
        chunk_size = 8192
        pattern = re.compile(r'<td style="width:40%"><b>([^<]+)</b></td>')
        buffer = ""
        
        # Process the response in chunks
        for chunk in response.iter_content(chunk_size=chunk_size, decode_unicode=True):
            if not chunk:
                continue
                
            # Add chunk to buffer and search for pattern
            buffer += chunk
            match = pattern.search(buffer)
            
            if match:
                return match.group(1)
                
            # Keep only the last 100 characters in case pattern spans chunks
            buffer = buffer[-100:] if len(buffer) > 100 else buffer
            
        return "Birth date not found"

# Example usage
results = search_athletes("jimmy gressier")
seq = results[0].get('seq')
# URL à scraper
url = f"https://bases.athle.fr/asp.net/athletes.aspx?base=bilans&seq={seq}"
birth_date = scrape_birth_date(url)
print("Birth date:", birth_date)

Birth date: 04/05/1997
